In [ ]:
!pip install ortools

from google.colab import files

uploaded = files.upload()

from ortools.linear_solver import pywraplp
import math
import numpy as np
import pandas as pd
import io

file_name = 'Copy of Fuguang-sample-input-lex.csv'

data_types = {'RackID': str}
df1 = pd.read_csv('Fuguang-sample-input-lex.csv', dtype=data_types)



Saving Fuguang-sample-input-lex.csv to Fuguang-sample-input-lex (2).csv


In [ ]:
# df1.drop(['RackID', 'UltraS_T', 'Rinse_T'],axis=1, inplace=True)
num_racks = len(df1)

def main():
  #Solver
  # Create the mip solver with the SCIP backend.
  solver = pywraplp.Solver('SolveSimpleSystem', pywraplp.Solver.SAT_INTEGER_PROGRAMMING)


  # Variables
  # x[i, j] is an array of 0-1 variables, which will be 1
  # if rack i is assigned to task j.
  x = {}
  for i in range(num_racks):
      for j in range(num_racks):
          x[i, j] = solver.IntVar(0, 1, '')

  # Constraints
  # 1. Each rack is assigned to at most 1 task.
  for i in range(num_racks):
      solver.Add(solver.Sum([x[i, j] for j in range(num_racks)]) <= 1)

  # 2. Each task is assigned to exactly one rack.
  for j in range(num_racks):
      solver.Add(solver.Sum([x[i, j] for i in range(num_racks)]) == 1)

  # constraint 3,4,5
  wt_p = []
  wt_tp = []
  wb_p = [0]
  wb_tp = []

  for i in range(num_racks):
      wt_p.append(solver.Sum([x[i, j] * df1['Prime_T'][j] \
                            for j in range(num_racks)]))
      wt_tp.append(solver.Sum([x[i, j] * df1['TopCoat_T'][j] \
                              for j in range(num_racks)]))
      wb_p.append(wb_p[i] + wt_p[i])

  for i in range(num_racks):
      wb_tp.append(wb_p[i+20] if i < num_racks - 20 else wb_tp[i-1] + wt_tp[i-1])
      wt_p[i] + wb_p[i] <= wb_tp[i]

  # 6. the completion time for the (i+1)th rack in prime tank
  wt_p[1] + wb_p[1] == wt_p[0] + wb_p[0] + max(df1['Prime_T'][0],df1['Prime_T'][1])
  for i in np.arange(1,19):
      wt_p[i+1] + wb_p[i+1] == wt_p[i] + wb_p[i] + max(df1['Prime_T'][i-1],\
                                                      df1['Prime_T'][i],df1['Prime_T'][i+1])

  wt_p[20] + wb_p[20] == wt_p[19] + wb_p[19] + max(df1['Prime_T'][18],\
                                                      df1['Prime_T'][19],df1['Prime_T'][20],df1['TopCoat_T'][0])

  wt_p[21] + wb_p[21] == wt_p[20] + wb_p[20] + max(df1['Prime_T'][19],\
                                                      df1['Prime_T'][20],df1['Prime_T'][21],df1['TopCoat_T'][0],df1['TopCoat_T'][1])

  for i in np.arange(21,num_racks-1):
      wt_p[i+1] + wb_p[i+1] == wt_p[i] + wb_p[i] + max(df1['Prime_T'][i-1],\
                                                      df1['Prime_T'][i],df1['Prime_T'][i+1],\
                              df1['TopCoat_T'][i-21],df1['TopCoat_T'][i-20],df1['TopCoat_T'][i-19])

  # 7. the completion time for the (i+21)th rack in prime tank
  # equals the completion time for the ith rack in Top Coat tank
  for i in np.arange(0,num_racks-20):
      wb_p[i+20] + wt_p[i+20] == wb_tp[i] + wt_tp[i]

  # Objective
  solver.Minimize(wb_tp[num_racks-1] + wt_tp[num_racks-1])

  # Solve
  status = solver.Solve()

  wt_p_new = []
  wt_tp_new = []
  y = []

  # Print solution.
  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
      for j in range(num_racks):
          for i in range(num_racks):
              # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
              if x[i, j].solution_value() > 0.5:
                  y.append(i)
                  #print('x[',i,',', j,']=',x[i,j].solution_value())
                  #print(f'rack {i} assigned to position {j}.')
                  #print('\n')
                  wt_p_new.append(df1['Prime_T'][i])
                  wt_tp_new.append(df1['Prime_T'][i])
                  continue
              else:
                  x[i,j] = x[i, j].solution_value()
      print('wt_p: {} \n wt_tp: {}'.format(wt_p_new, wt_tp_new))
  else:
      print('No solution found.')

  wb_p_new = [None] * num_racks
  wb_p_new[0] = 0

  wb_tp_new = [None] * num_racks


  for j in range(num_racks-1):
      wb_p_new[j+1] = wb_p_new[j] + wt_p_new[j]
      #print('wb_p_new [', j, ']= ', wb_p_new[j])

  for j in range(num_racks-20):
      wb_tp_new[j] = wb_p_new[j+20]

  for j in range(num_racks-20,num_racks):
      wb_tp_new[j] = wb_tp_new[j-1] + wt_tp_new[j-1]
  #for j in range(num_racks):
  #    wb_tp_new[j] = wb_p_new[j+20] if j < num_racks-20 else wb_tp_new[j-1] + wt_tp_new[j-]

  #print(wb_tp_new)
  #print('The total completion time: ', (wb_tp_new[num_racks-1]+wt_tp_new[num_racks-1])/3600, 'hours.')

  start_tank_distance = 25
  TANK_RACK_DISTANCE = 20
  RAMP_TANK_DISTANCE = 34
  pretanktime = start_tank_distance * min(wt_p_new[i] for i in range(start_tank_distance))
  postPrimeTime = RAMP_TANK_DISTANCE * min(wt_tp_new[i] for i in range(130,num_racks))

  return [y, (wb_tp_new[num_racks-1] + wt_tp_new[num_racks-1] + pretanktime + postPrimeTime)]

if __name__ == '__main__':


    [y, run_time]= main()



wt_p: [204, 244, 244, 374, 243, 277, 204, 282, 243, 243, 243, 244, 299, 257, 204, 211, 194, 277, 274, 204, 191, 274, 282, 194, 194, 191, 243, 299, 244, 243, 374, 244, 374, 244, 191, 194, 274, 194, 210, 194, 277, 244, 243, 243, 244, 194, 282, 256, 244, 274, 243, 191, 286, 256, 299, 203, 210, 274, 214, 299, 214, 204, 244, 256, 243, 256, 194, 274, 299, 244, 204, 203, 244, 203, 299, 274, 277, 286, 243, 191, 282, 210, 257, 243, 203, 204, 243, 210, 244, 244, 277, 203, 256, 244, 256, 299, 244, 277, 203, 211, 257, 244, 243, 243, 243, 210, 194, 243, 203, 204, 286, 243, 243, 211, 282, 243, 299, 210, 277, 194, 299, 204, 277, 282, 243, 204, 244, 243, 277, 204, 194, 274, 286, 277, 243, 277, 299, 244, 191, 211, 243, 243, 286, 285, 277, 282, 299, 204, 214, 282, 286, 244, 274, 244, 194, 214, 210, 243, 282, 277, 204, 299] 
 wt_tp: [204, 244, 244, 374, 243, 277, 204, 282, 243, 243, 243, 244, 299, 257, 204, 211, 194, 277, 274, 204, 191, 274, 282, 194, 194, 191, 243, 299, 244, 243, 374, 244, 374, 244, 191

# New Section

In [ ]:
#[y, run_time]= main()
print('y = ', y)
print('Schedule Rack Count: ', len(df1), 'racks')
print('Completion time = ', round(run_time/3600,2), 'hours')


y =  [140, 1, 161, 2, 4, 64, 5, 90, 6, 146, 0, 3, 9, 101, 70, 102, 7, 150, 12, 49, 14, 82, 143, 92, 13, 84, 104, 17, 18, 142, 20, 103, 80, 158, 100, 21, 10, 23, 112, 25, 107, 16, 156, 24, 27, 111, 15, 148, 11, 108, 144, 31, 145, 33, 22, 81, 160, 39, 34, 29, 114, 106, 72, 42, 36, 113, 40, 35, 50, 41, 93, 26, 46, 153, 152, 121, 43, 51, 44, 38, 19, 105, 47, 48, 110, 109, 58, 75, 118, 67, 129, 155, 94, 128, 149, 56, 30, 116, 157, 45, 135, 147, 120, 57, 37, 127, 136, 63, 55, 125, 99, 124, 95, 68, 88, 133, 52, 119, 59, 138, 73, 32, 8, 60, 69, 115, 132, 130, 53, 61, 62, 96, 77, 71, 28, 141, 91, 79, 139, 117, 134, 78, 86, 66, 83, 85, 74, 65, 151, 159, 76, 137, 154, 122, 89, 131, 87, 54, 97, 123, 126, 98]
Schedule Rack Count:  162 racks
Completion time =  15.58 hours


In [ ]:
df2 = df1.copy()
df2.tail(40)
completion_time(df2, y)/3600

17.321944444444444

In [ ]:
import random
!pip install iteration_utilities
from iteration_utilities import duplicates

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iteration_utilities: filename=iteration_utilities-0.11.0-cp310-cp310-linux_x86_64.whl size=327852 sha256=837f8de660b8f09ab1d84a575e9a6e126b12bc2bb290b61ff969bcba65910e5a
  Stored in directory: /root/.cache/pip/wheels/2d/ab/c4/309aa5942beba1f36d6744acf619b7a314fca50f6a7ccce68c
Successfully built iteration_utilities


In [ ]:
def Generate_Initial_Population(problem_size, population_size) -> list:
    population = []

    for i in range (population_size):

        # create list with size == problem size and random values ranging from 0 to problem_size
        x = random.sample(range(problem_size), problem_size)

        # add list x to population. The item in the second index (0) is the fitness score we'll use later
        population.append([x, 0])

    return population

start_tank_distance = 25
TANK_RACK_DISTANCE = 20
RAMP_TANK_DISTANCE = 34

def reorder(mydf_old, chromosome):
    '''
    mydf_old['RackID'] = pd.Categorical(mydf_old['RackID'], categories=chromosome, ordered=True)
    mydf_old = mydf_old.sort_values('RackID')
    #df1['RackID'] = df1['RackID'].astype(int)
    mydf_old.drop(['RunID'], axis=1, inplace=True)
    mydf_old.insert(loc = 0,
          column = 'RunID',
          value = [i for i in range(1,len(df1)+1)])
    mydf_reset = mydf_old.reset_index(drop=True)
    return mydf_reset

    '''
    df2 = mydf_old.copy()

    df2['RackID'] = df2['RackID'].astype(int)

    # Define the desired permutation of 'y'
    permutation_of_y = chromosome

    # Apply the permutation to reorder the DataFrame
    df_new = df2.iloc[permutation_of_y]

    return df_new

def completion_time(df_old, chromosome):

  df_new= reorder(df_old, chromosome)

  time = 0
  # First 25 racks move at min Prime time
  pretanktime = start_tank_distance * min(df_new['Prime_T'][i] for i in range(start_tank_distance))

  time = df_new['Prime_T'][0] + max(df_new['Prime_T'][0],df_new['Prime_T'][1]) # rack 0, 1 in tank 26

  for i in range(2, 20): # rack 2 - 19 in tank 26
      time += max(df_new['Prime_T'][i-2], df_new['Prime_T'][i-1], df_new['Prime_T'][i])

  # RackID 20 in tank 26
  time += max(df_new['Prime_T'][18], df_new['Prime_T'][19], df_new['Prime_T'][20], df_new['TopCoat_T'][0])
  # RackID 21 in tank 26
  time += max(df_new['Prime_T'][19], df_new['Prime_T'][20], df_new['Prime_T'][21], df_new['TopCoat_T'][0], df_new['TopCoat_T'][1])

  for i in range(0, num_racks-22): # Rack 22-num_racks-1 in tank 26, that is, rack 2-num_racks-21 in tank 46
      time += max(df_new['TopCoat_T'][i], df_new['TopCoat_T'][i+1], df_new['TopCoat_T'][i+2], df_new['Prime_T'][i+20], df_new['Prime_T'][i+21], df_new['Prime_T'][i+22])

  # the last RackID num_racks-1 in tank 27
  time += max(df_new['TopCoat_T'][num_racks-22], df_new['TopCoat_T'][num_racks-21], df_new['TopCoat_T'][num_racks-20], df_new['Prime_T'][num_racks-2], df_new['Prime_T'][num_racks-1])

  # the last part RackID num_racks-1 in tank 28
  time += max(df_new['TopCoat_T'][num_racks-21], df_new['TopCoat_T'][num_racks-20], df_new['TopCoat_T'][num_racks-19], df_new['Prime_T'][num_racks-1])

  for i in range(num_racks-20, num_racks-2): # All racks are in TopCoat tanks only
      time += max(df_new['TopCoat_T'][i], df_new['TopCoat_T'][i+1], df_new['TopCoat_T'][i+2])

  # the last part RackID num_racks-1 in rack 47
  time += max(df_new['TopCoat_T'][num_racks-2], df_new['TopCoat_T'][num_racks-1])

  # the last part RackID num_racks-1 in rack 48
  time += df_new['TopCoat_T'][num_racks-1]

  # Last 34 racks move at min TC time
  postPrimeTime = RAMP_TANK_DISTANCE * min(df_new['TopCoat_T'][i] for i in range(130,num_racks))

  return time + pretanktime + postPrimeTime


def fitness(df, population):

  for i in range(len(population)):
      #if i < 4:
      population[i][1] = completion_time(df, population[i][0])

  return population

def Selection_Function(population) -> list:
    #random.seed(1)
    # return random list size population_size/2 from population
    random_k_list = random.sample(population, int(len(population)/2))

    # sort random list using their fitness score
    random_k_list.sort(key = lambda x: x[1])

    # return first element after sort
    return random_k_list[0]

def Crossover_Function(data1, data2):

    # for this function, I modified the uniform crossover function to take care of duplicates after crossover.

    data1[1] = 0
    data2[1] = 0
    chromosome1 = data1[0].copy()
    chromosome2 = data2[0].copy()


    # for each index in both chromosomes, use a coin toss to determine which index is crossed over
    for i in range(len(chromosome1)):

        cointoss = random.randrange(2)
        if cointoss == 0:
            chromosome1[i], chromosome2[i] = chromosome2[i], chromosome1[i]

    # find duplicates after crossing over
    dupes_in_ch1 = list(duplicates(chromosome1))
    dupes_in_ch2 = list(duplicates(chromosome2))


    # handle duplicates if any are found
    for i in dupes_in_ch1:
        if i in chromosome1: chromosome1.remove(i)
        chromosome2.append(i)

    for i in dupes_in_ch2:
        if i in chromosome2: chromosome2.remove(i)
        chromosome1.append(i)

    # replaced the modified chromosomes in the data
    data1[0] = chromosome1
    data2[0] = chromosome2

    data1[1] = completion_time(df1, chromosome1)
    data2[1] = completion_time(df1, chromosome2)

    return [data1, data2]

def Mutation_Function(data) -> list:

    chromosome = data[0]

    randomNum1 = random.randint(0, len(chromosome) - 1)
    randomNum2 = random.randint(0, len(chromosome) - 1)

    # exchange values at 2 random indexes
    chromosome[randomNum1], chromosome[randomNum2] = chromosome[randomNum2], chromosome[randomNum1]

    data[1] = completion_time(df1, chromosome)

    return data

import sys

def GeneticAlgorithm(problem_size, population_size, mydf, number_of_iterations):

    # Get dictionary that generates random value for dip time between locations and PartStyles

    # generate initial population
    population = Generate_Initial_Population(problem_size, population_size)
    population.append([y, run_time])
    #print(population)

    solution = int(sys.maxsize)
    next_generation = []
    n = 0


    while n < number_of_iterations:

        # get cost function for each data in population
        population = fitness(mydf, population)

        # sort population according to fitness score
        population.sort(key = lambda x: x[1])

        # get fittest data
        fittest_data = population[0].copy()

        # check for the fittest data and print it out
        if fittest_data[1] < solution:
            result = fittest_data.copy()
            solution = fittest_data[1]
            #print("\nSolution for iteration - " + str(n))
            #print(result)


        while len(next_generation) < len(population):

            # use selection fucntion to get 2 fit chromosomes
            data1 = Selection_Function(population)
            data2 = Selection_Function(population)

            # crossover the 2 chromosome
            crossed_over_data = Crossover_Function(data1, data2)
            #print('crossed_over_data = ', crossed_over_data)

            # mutate both chromosomes
            offspring1 = Mutation_Function(crossed_over_data[0])
            offspring2 = Mutation_Function(crossed_over_data[1])

            # add offsprings to next generation
            next_generation.append(offspring1)
            next_generation.append(offspring2)

            #print('offspring1 = , offspring2 = ', offspring1, offspring2)

        # repeat iteration with new generation
        population = next_generation
        next_generation = []
        n+=1


    #print final result
    #print("Final solution after " + str(n) +" iterations = ")
    #print(result)

    return result



In [ ]:
import random

def Mutation_Function(data) -> list:

    chromosome = data[0]

    randomNum1 = random.randint(0, len(chromosome) - 1)
    randomNum2 = random.randint(0, len(chromosome) - 1)

    # exchange values at 2 random indexes
    chromosome[randomNum1], chromosome[randomNum2] = chromosome[randomNum2], chromosome[randomNum1]

    data[1] = completion_time(df1, chromosome)

    return data

mydf_old = df1

df2 = mydf_old.copy()

df2['RackID'] = df2['RackID'].astype(int)

# Define the desired permutation of 'y'
permutation_of_y = y

# Apply the permutation to reorder the DataFrame
df_new = df2.iloc[permutation_of_y]
#print(df2.head(10))
df3 = reorder(df2, y)
print(df3.head(10))
'''
df_new.head(10)
'''
y1 = Mutation_Function([y,0])
if y1[0] == y:
  print(True)
else:
  print(False)
#completion_time(df2, y1[0])

     RackID PartID  HandID  nParts  UltraS_T  Rinse_T  Prime_T  TopCoat_T  \
140      19    5A8       6     447       230      205      204        240   
1       110    609       2     210       257      160      244        235   
161     135    609       1     227       257      160      244        235   
2        23    631       6      29       245      277      374        195   
16       54    609       1     227       257      160      244        235   
64       75    588       2     420       246      144      277        217   
5        47    5A8       6     262       230      205      204        240   
90      108    588       0      76       281      144      282        282   
6       101    654       4     382       245      178      243        215   
146     127    654       3     652       245      178      243        215   

     Unload_T  RackDensity  nRacksExact  
140      1920           91     4.912088  
1         540           42     5.000000  
161       540           42

In [ ]:

problem_size, population_size, df, number_of_iterations = len(df1), 4, df1, 100
result = GeneticAlgorithm(problem_size, population_size, df, number_of_iterations)
print('Schedule Rack Count: ', len(df1), 'racks')
print('Run time is: ', round(result[1]/3600,2), 'hours')
#print(result[0])

Schedule Rack Count:  162 racks
Run time is:  17.32 hours


In [ ]:
mydf = reorder(df1,result[0])
#mydf.head(10)
mydf.to_csv('10000.csv', encoding = 'utf-8-sig', index=False)
files.download('10000.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
completion_time(df1, result[0])

91377

In [ ]:
pretanktime = start_tank_distance * min(mydf['Prime_T'][i] for i in range(start_tank_distance))
postPrimetime = RAMP_TANK_DISTANCE * min(mydf['TopCoat_T'][i] for i in range(130,164))
print('pretanktime, postprimetime = ', pretanktime, postPrimetime)

In [ ]:
# 91602, 5000
# 91368, 10000
# pretanktime, postprimetime =  5250 6596